### Data Load & Setting

In [5]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel
from keybert import KeyBERT
import ast

/Users/gimtaeho/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
data=pd.read_csv('../data/menu_v3_sample_sentence.csv')

# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_name_split"] = data["menu_name_split"].apply(ast.literal_eval)
data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rst_name               40 non-null     object
 1   review_sentence_split  40 non-null     object
 2   menu_name_split        40 non-null     object
 3   org_menu_dict          40 non-null     object
 4   menu_sentence          40 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [8]:
data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"[, 족발 하 고, 불 족발 새우젓 마늘 쌈장 쌈 야채, 실속 알뜰 보쌈 겉절이, ...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[뼈탕, 양념 주물럭, 모듬한마리, 양념주물럭, 오리 주물럭, 훈제오리, 모듬 한 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 고기시키면 뼈탕이 서비스였는데 들..."
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[토시 살, 된장찌개 밥, 가브리 살, 소고기 세트, 된장찌개밥, 돼지세트 , 김치...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 고기도 정말 맛있..."
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","[왕 계 이 란 말 이, 고기반김치찌개세트, 고기 반 김치찌개 세트, 왕계란말이, ...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","[김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요, 퐁..."
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","[물회, 새꼬막양념, 벌교참꼬막, 간 재미, 새 꼬막, 참 꼬막 양념, 간재미 회무...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","[벌교참꼬막 38000 2인분 정도 될듯해요, 굴회무침 30 000 간재미무침과 양..."


### Functions

In [9]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith("N") or pos.startswith("SL")):
            results.append(token)
    return results

# 명사 추출 함수
def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results

# 명사 추출 함수
def extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if (
            len(token) != 1
            and pos.startswith("N")
            or pos.startswith("SL")
            or pos.startswith("V")
        ):
            results.append(token)
    return results

In [10]:
import ast

# 문자열 형태의 리스트를 파이썬 리스트로 변환
# data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인

### KeyBERT to DataFrame (by. Moonsoo)

In [11]:
# KeyBERT 로드. (KoBERT 사용)

model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [12]:
# keybert돌리고 키워드 리턴하는 함수 (!! -- ngram은 무조건 켜야함 -- !!)
def extract_keywords_from_reviews_candy(document,candidate,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1,3),  # 단어 n-gram 범위
        stop_words=None,  # 불용어F
        # use_maxsum=False,
        # use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        # highlight=True,
        candidates=candidate
    )  # 상위 n개 키워드
    # print(f"Keywords: {keywords}\n")

    return keywords

# 데이터 형식을 input text에 맞게 변환하는 함수 제작 (for문 안에서 돌던거를 위로 뺐음)
def doc_to_input_text(review_doc):
    # 식당의 문장 리스트 추출
    input_text = ".".join(review_doc)
    input_text = adverb_remover(input_text)
    input_text=" ".join(input_text)

    return input_text

In [13]:
# input text 열 만들기
data['input_text'] = data['menu_sentence'].apply(doc_to_input_text)

In [14]:
# 데이터 생긴거 확인 가능

data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence,input_text
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"[, 족발 하 고, 불 족발 새우젓 마늘 쌈장 쌈 야채, 실속 알뜰 보쌈 겉절이, ...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...,잡내 맛있 구의 먹자골목 바로 근성 매우 지나가 자주 서비스 메뉴 알차 맵 .. 아...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[뼈탕, 양념 주물럭, 모듬한마리, 양념주물럭, 오리 주물럭, 훈제오리, 모듬 한 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 고기시키면 뼈탕이 서비스였는데 들...",미나리 같이 오리 주물럭 아주 맛있 고기 시키 뼈탕 서비스 들깨 국물 맛있 .. 도...
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[토시 살, 된장찌개 밥, 가브리 살, 소고기 세트, 된장찌개밥, 돼지세트 , 김치...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 고기도 정말 맛있...",토시 갈비 섯구이 구성 소고기 세트 주문 고기 정말 맛있 사이드 주문 쫄면 된장찌개...
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","[왕 계 이 란 말 이, 고기반김치찌개세트, 고기 반 김치찌개 세트, 왕계란말이, ...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","[김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요, 퐁...",김찌 맵기 조절 가능 칼칼하 제육 짭졸 퐁실퐁실 진짜 부드럽 쌉꿀마 김치찌개 계란말...
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","[물회, 새꼬막양념, 벌교참꼬막, 간 재미, 새 꼬막, 참 꼬막 양념, 간재미 회무...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","[벌교참꼬막 38000 2인분 정도 될듯해요, 굴회무침 30 000 간재미무침과 양...",벌교 꼬막 38000 인분 정도 굴회 무침 30 000 재미 무침 양념 상큼 음식 ...


In [15]:
rst_name_list, menu_name_list, keybert_scores = [], [], []
cols = ["rst_name", "main_menu", "keybert_score", "reviews" ,"org_menu_dict"]
seulbeen_output=[]
for i in range(40):
    # 식당의 문장 리스트 추출
    
    input_text = data.loc[i, 'input_text']
    menu_candidates=data.loc[i, 'menu_name_split']
    org_menu_dict=data.loc[i,'org_menu_dict']

    review = data.loc[i,'review_sentence_split']
    rst_name = data.loc[i, 'rst_name']

    results = extract_keywords_from_reviews_candy(input_text,menu_candidates)
    print(results)
    if results:
        for result in results:
            rst_info = []
            rst_info.append(rst_name)
            name, score = result[0], result[1]
            rst_info.append(name)
            rst_info.append(score)
            rst_info.append(review)
            rst_info.append(org_menu_dict)
            seulbeen_output.append(rst_info)
        del [[rst_info]]
    else:
        print(f"PASS")

[]
PASS
[('오리 주물럭', 0.4592), ('양념 주물럭', 0.4182), ('뼈탕', 0.163)]
[('오겹살', 0.5143), ('목살', 0.4756), ('매콤', 0.4726), ('김치찌개', 0.4596), ('가브리살', 0.4242), ('고공 세트', 0.3255), ('소고기 세트', 0.3055), ('돼지 세트', 0.2996), ('된장찌개', 0.281)]
[('제육', 0.4742), ('간장 제육', 0.3241)]
[('쭈꾸미 구이', 0.7002), ('짱뚱어탕', 0.6895), ('물회', 0.3199)]
[('야채 곱창', 0.4808), ('곱창', 0.4687)]


/Users/gimtaeho/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


[('미나리', 0.604), ('양지', 0.5579), ('양지 갈비', 0.5442), ('양지 갈비탕', 0.5432), ('양지 갈비 쌀국수', 0.4823), ('얼큰 양지', 0.4316), ('얼큰', 0.3879), ('양지 쌀국수', 0.3701), ('모둠 쌀국수', 0.3679), ('얼큰 양지 소고기', 0.3238), ('모둠', 0.187)]
[('빠네 파스타', 0.603), ('까르보나라', 0.4862), ('마라 크림', 0.4429), ('치킨 크림 리조또', 0.314), ('치즈 오븐 스파게티', 0.3117), ('파스타', 0.2989), ('목살 스테이크', 0.2315)]
[('수육', 0.2057)]
[('내장 곰탕', 0.5306), ('도가니탕', 0.4795), ('설렁탕', 0.4458), ('도가니 수육', 0.3726), ('갈비탕', 0.3591), ('꼬리 곰탕', 0.3461)]
[('김치전', 0.6207), ('칼제비', 0.4423), ('파전', 0.3493), ('도토리묵', 0.3296), ('항아리 수제비', 0.2949), ('콩국수', 0.2845), ('민속 국시', 0.2439)]
[('바지락 칼국수', 0.3495)]
[('소고기 샤브샤브', 0.7158), ('한우 샤브샤브', 0.6818), ('한우', 0.5087), ('소고기', 0.4156), ('소고기 해물', 0.3982)]
[('보배 짜장면', 0.6314), ('새우 고추 짬뽕', 0.5431), ('해장', 0.4792), ('보배 짬뽕', 0.3584), ('크림 짬뽕', 0.3143), ('해물 쟁반 짜장', 0.3012), ('소고기 짬뽕', 0.2737), ('해물 짬뽕', 0.2545)]
[]
PASS
[('시오버터 라멘', 0.5461), ('산쪼메 라멘', 0.5103), ('츠케멘', 0.4586), ('카라카라돈코츠라멘', 0.4262), ('돈코츠 라멘', 0.3755), ('카라카라', 

/Users/gimtaeho/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


[('양념 통닭', 0.1222), ('닭똥집', 0.1063)]
[('추가', 0.1001)]
[('바샤샥', 0.582), ('바샤샥해물전', 0.5431), ('꼬막 소면', 0.5206), ('꼬막', 0.4778)]
[('감자전', 0.6118), ('막국수', 0.4557), ('수육', 0.2438)]


...

[('새우 튀김', 0.5833), ('서더리탕', 0.4078), ('참돔', 0.4006), ('세트', 0.3473), ('해산물', 0.3159)]
[]
PASS
[('고사리 표고', 0.6219), ('한우 차돌박이', 0.5912), ('미나리 꼬막', 0.5516), ('차돌박이', 0.5439), ('미나리', 0.5103), ('미나리 꼬막 무침', 0.482), ('한우', 0.4737), ('고사리', 0.3779), ('곤드레 토마토', 0.3089), ('매콤', 0.2982), ('곤드레', 0.2865)]
[('고등어조림', 0.5495), ('고등어 조림', 0.5448), ('고등어 구이', 0.492), ('성게 미역국', 0.4023), ('세트메뉴', 0.334), ('세트 메뉴', 0.3234)]
[('한우', 0.5989), ('곱창 전골', 0.5672), ('곱창전골', 0.5321)]
[('지존 짬뽕', 0.638), ('지존 짜장', 0.6132), ('수제비 짬뽕', 0.6114), ('찹쌀 탕수육', 0.5412), ('게살 볶음밥', 0.5086), ('순두부', 0.5026), ('수제비', 0.487), ('쌀국수 짬뽕', 0.4825), ('사천 탕수육', 0.3432)]
[('스페셜', 0.353)]


/Users/gimtaeho/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


[('생신 잔치', 0.3986), ('상견례', 0.3976), ('한정식', 0.2964)]
[('뿌팟퐁 커리', 0.5459), ('뿌팟퐁커리', 0.4702), ('쁠라텃', 0.4501), ('뿌팟퐁', 0.4347), ('뿌팟퐁 커리 덮밥', 0.3498), ('뿌팟퐁커리덮밥', 0.2611), ('무텃', 0.2552), ('태국 쌀국수', 0.236), ('똠양 쌀국수', 0.1652)]
[('최루탄 주먹밥', 0.3437)]
[('한우', 0.4899), ('한우 세트', 0.2836)]


/Users/gimtaeho/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


[('짜장면', 0.7136), ('쟁반 짜장', 0.5853), ('탕수육', 0.5485), ('해물 짬뽕', 0.4478)]


In [16]:
seulbeen_output_df=pd.DataFrame(seulbeen_output,columns=cols)

In [17]:
seulbeen_output_df.head(20)

,rst_name,main_menu,keybert_score,reviews,org_menu_dict
0,강나루 유황오리주물럭 본점,오리 주물럭,0.4592,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모..."
1,강나루 유황오리주물럭 본점,양념 주물럭,0.4182,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모..."
2,강나루 유황오리주물럭 본점,뼈탕,0.1630,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모..."
3,고공 구의점,오겹살,0.5143,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
4,고공 구의점,목살,0.4756,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
5,고공 구의점,매콤,0.4726,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
6,고공 구의점,김치찌개,0.4596,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
7,고공 구의점,가브리살,0.4242,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
8,고공 구의점,고공 세트,0.3255,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
9,고공 구의점,소고기 세트,0.3055,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."


In [18]:
# seulbeen_output_df.to_csv("../data/rst_menu_keybert_score.csv",encoding='utf-8-sig')

### 감성분석 점수 (by. Taeho)

org_menu_dict에서 main_menu를 포함하는 딕셔너리만 추출
-> filtered_org_menu_dict 생성

filtered_org_menu_dict의 value를 포함하는 문장만 추출
-> filtered_reviews 생성

filtered_reviews의 각 문장들 감성점수 평균 계산 (긍정:양수 / 부정:음수)
-> senti_score 추출

In [49]:
# import library
import torch
import pandas as pd
import ast
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

data = seulbeen_output_df
# data.head()

In [50]:
# main menu로 filtered_org_menu_dict 생성
def filter_org_menu_dict(row):
    # 문자열로 표기된 딕셔너리 실제 딕셔너리 리스트로 변환
    org_menu_dicts = ast.literal_eval(row['org_menu_dict'])
    main_menu = row['main_menu']

    # value에 main_menu 포함하는 org_menu_dicts만 필터링
    filtered_org_menu_dicts = []
    for menu_dict in org_menu_dicts:
        for key, variations in menu_dict.items():
            if main_menu in variations:
                filtered_org_menu_dicts.append(menu_dict)
                break
    
    return filtered_org_menu_dicts


# filtered_org_menu_dict로 filtered_reviews 생성
def filter_reviews_by_menu(row):
    # 리뷰 문자열을 리스트로 변환
    reviews = ast.literal_eval(row['reviews'])
    
    # filtered_org_menu_dict로부터 추출된 모든 메뉴(value) 추출
    menu_variations = set()
    for menu_dict in row['filtered_org_menu_dict']:
        for variations in menu_dict.values():
            menu_variations.update(variations)
    
    # menu_variations 포함하는 리뷰만 필터링

    filtered_reviews = [review for review in reviews if any(menu_variation in review for menu_variation in menu_variations)]
    
    return filtered_reviews

# filter_org_menu_dict 적용
data['filtered_org_menu_dict'] = data.apply(filter_org_menu_dict, axis=1)

# filter_reviews_by_menu 적용
data['filtered_reviews'] = data.apply(filter_reviews_by_menu, axis=1)

# 결과 출력
data[['main_menu', 'org_menu_dict', 'filtered_org_menu_dict', 'reviews', 'filtered_reviews']].head()


,main_menu,org_menu_dict,filtered_org_menu_dict,reviews,filtered_reviews
0,오리 주물럭,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}]","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 찌니가좋아하는 오리주물럭 정말 맛..."
1,양념 주물럭,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'양념주물럭': ['양념 주물럭', '양념주물럭']}]","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...",[양념주물럭 정말 맛있었어요]
2,뼈탕,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'뼈탕': ['뼈탕', '뼈 탕']}]","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[고기시키면 뼈탕이 서비스였는데 들깨향나는 국물이 맛있었습니다 ., 다 왕맛도리 부..."
3,오겹살,"[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[{'오겹살': ['오 겹 살', '오겹살']}]","['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...",[오겹살 진짜 맛도리 기본찬도 정갈하고 김치 하나하나 맛있어서 친구랑 다음에 또 ...
4,목살,"[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[{'목살': ['목살']}],"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[그 중 목살이 태어나서 먹어본 목살 중 제일 맛있었어요, 목살 가브리살 오겹살..."


In [51]:
# load model
tokenizer = AutoTokenizer.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

In [53]:
# filtered_reviews로 senti_score 계산

# 감성분석 함수 (긍정은 양수, 부정은 음수)
def get_sentiment_score(sentence):
    result = sentiment_classifier(sentence)[0]
    return result['score'] if result['label'] == '1' else -result['score']

# 여러 문장의 평균 감성 점수를 계산하는 함수
def get_average_sentiment_score(sentences):
    scores = [get_sentiment_score(sentence) for sentence in tqdm(sentences, desc="Processing sentences")]
    return sum(scores) / len(scores) if scores else 0

# 'reviews' 열의 문장 리스트를 사용하여 평균 감성 점수 계산
data['senti_score'] = data['filtered_reviews'].apply(get_average_sentiment_score)

Processing sentences: 100%|██████████| 2/2 [00:00<00:00, 16.67it/s]


In [54]:
data.head()

,rst_name,main_menu,keybert_score,reviews,org_menu_dict,filtered_reviews,senti_score,filtered_org_menu_dict
0,강나루 유황오리주물럭 본점,오리 주물럭,0.4592,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 찌니가좋아하는 오리주물럭 정말 맛...",0.944607,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}]"
1,강나루 유황오리주물럭 본점,양념 주물럭,0.4182,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...",[양념주물럭 정말 맛있었어요],0.990226,"[{'양념주물럭': ['양념 주물럭', '양념주물럭']}]"
2,강나루 유황오리주물럭 본점,뼈탕,0.1630,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[고기시키면 뼈탕이 서비스였는데 들깨향나는 국물이 맛있었습니다 ., 다 왕맛도리 부...",0.979921,"[{'뼈탕': ['뼈탕', '뼈 탕']}]"
3,고공 구의점,오겹살,0.5143,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[오겹살 진짜 맛도리 기본찬도 정갈하고 김치 하나하나 맛있어서 친구랑 다음에 또 ...,0.762647,"[{'오겹살': ['오 겹 살', '오겹살']}]"
4,고공 구의점,목살,0.4756,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[그 중 목살이 태어나서 먹어본 목살 중 제일 맛있었어요, 목살 가브리살 오겹살...",0.792429,[{'목살': ['목살']}]
5,고공 구의점,매콤,0.4726,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[사이드메뉴로 매콤쫄면은 주문 필수 완전 프레시하고 입가심되고 고기랑같이먹기 좋았...,0.520867,"[{'매콤 쫄면': ['매콤 쫄 면', '매콤', '매콤 쫄면', '매콤쫄면']}]"
6,고공 구의점,김치찌개,0.4596,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[마지막으로 식사 메뉴 김치찌개까지 야무지게 먹고 왔어용, 김치찌개도 진하고 고기도...",0.364836,"[{'김치찌개+밥': ['김치찌개밥', '김치찌개', '김치찌개 밥']}]"
7,고공 구의점,가브리살,0.4242,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[목살 가브리살 오겹살 먹었습니다, 가브리살이 엄청 부드럽고 목살도 너무 맛있어...",0.895991,"[{'가브리살': ['가브리살', '가브리 살']}]"
8,고공 구의점,고공 세트,0.3255,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[고공세트 85000 새로 5000 유자하이볼8000 된찌 6000 밥 버섯추가3...,0.980581,"[{'고공세트': ['고공세트', '고공 세트']}]"
9,고공 구의점,소고기 세트,0.3055,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 소고기세트는 토시...",0.839307,"[{'소고기세트': ['소고기 세트', '소고기세트']}]"


In [55]:
data.to_csv("../data/senti_score.csv",encoding='utf-8-sig')

### 데이터프레임 저장